In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_03 = pd.read_csv(folderpath + 'data_op_03.csv')
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,0.03,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,0.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,0.03,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,0.03,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,0.03,3,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_03 = tf_remove_sensor_15.fit_transform(data_op_03)
data_op_03.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
1,1,2,0.0015,0.0003,100.0,1584.55,1403.96,14.62,2388.01,521.81,2388.15,3,1
7,1,8,0.0011,0.0000,100.0,1589.55,1407.59,14.62,2388.10,521.74,2388.03,3,1
8,1,9,0.0011,0.0020,100.0,1586.18,1399.01,14.62,2388.10,521.72,2388.08,3,1
16,1,17,0.0003,0.0000,100.0,1584.57,1410.11,14.62,2388.06,522.17,2388.10,3,1
19,1,20,0.0004,0.0017,100.0,1581.33,1399.44,14.62,2388.09,521.62,2388.06,3,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_03)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
1,1584.55,1403.96,14.62,2388.01,521.81,2388.15,1
7,1589.55,1407.59,14.62,2388.10,521.74,2388.03,1
8,1586.18,1399.01,14.62,2388.10,521.72,2388.08,1
16,1584.57,1410.11,14.62,2388.06,522.17,2388.10,1
19,1581.33,1399.44,14.62,2388.09,521.62,2388.06,1


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-1.040896  , -0.62635123, -1.        , -1.37994051,  0.63774241,
         0.65278365],
       [-0.19248051, -0.21074616, -1.        , -0.05996073,  0.5391007 ,
        -1.09204307],
       [-0.76431255, -1.19308541, -1.        , -0.05996073,  0.51091736,
        -0.36503194],
       ...,
       [ 0.79168145,  0.75556424, -1.        ,  1.55334789, -1.33509174,
         2.39761038],
       [ 1.7639656 ,  1.08530049, -1.        ,  2.4333344 , -2.16650042,
         2.97921928],
       [ 1.69609236,  1.87872835, -1.        ,  2.4333344 , -1.67329188,
         1.81600147]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))}
                  }

In [13]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=50, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=394 ..........................................
[CV] hidden_layer_sizes=261 ..........................................
[CV] hidden_layer_sizes=261 ..........................................
[CV] hidden_layer_sizes=261 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed: 30.8min


[CV] . hidden_layer_sizes=759, score=0.6446043165467625, total=31.3min
[CV] hidden_layer_sizes=926 ..........................................
[CV] . hidden_layer_sizes=789, score=0.6235632183908046, total=16.2min
[CV] hidden_layer_sizes=926 ..........................................
[CV] . hidden_layer_sizes=150, score=0.6183644189383071, total=14.3min
[CV] hidden_layer_sizes=913 ..........................................
[CV] . hidden_layer_sizes=150, score=0.6255380200860832, total=13.7min
[CV] hidden_layer_sizes=913 ..........................................
[CV] .. hidden_layer_sizes=759, score=0.632183908045977, total=31.8min
[CV] hidden_layer_sizes=913 ..........................................
[CV] . hidden_layer_sizes=759, score=0.6097560975609756, total=32.0min
[CV] hidden_layer_sizes=913 ..........................................
[CV] .. hidden_layer_sizes=759, score=0.629842180774749, total=32.0min
[CV] hidden_layer_sizes=913 ..........................................
[CV] .

[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed: 68.1min


[CV] hidden_layer_sizes=794 ..........................................
[CV] . hidden_layer_sizes=926, score=0.6354466858789626, total=35.9min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=913, score=0.6446043165467625, total=35.1min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=791, score=0.6244604316546762, total=24.7min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=373, score=0.6054519368723099, total=24.5min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=667, score=0.6384505021520803, total=31.1min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=667, score=0.6350574712643678, total=31.2min
[CV] hidden_layer_sizes=512 ..........................................
[CV] . hidden_layer_sizes=791, score=0.6152737752161384, total=25.3min
[CV] h

[Parallel(n_jobs=100)]: Done 402 out of 500 | elapsed: 104.1min remaining: 25.4min


[CV] .. hidden_layer_sizes=97, score=0.6126255380200861, total= 7.4min
[CV] . hidden_layer_sizes=918, score=0.6397694524495677, total=24.4min
[CV] . hidden_layer_sizes=918, score=0.6460431654676259, total=24.9min
[CV] . hidden_layer_sizes=918, score=0.6123919308357348, total=25.0min
[CV] . hidden_layer_sizes=452, score=0.6325648414985591, total=14.8min
[CV] . hidden_layer_sizes=944, score=0.6340057636887608, total=20.1min
[CV] .. hidden_layer_sizes=97, score=0.6158273381294964, total= 7.3min
[CV] . hidden_layer_sizes=452, score=0.6212338593974175, total=16.9min
[CV] .. hidden_layer_sizes=97, score=0.6422413793103449, total= 8.4min
[CV] . hidden_layer_sizes=944, score=0.6054519368723099, total=23.0min
[CV] .. hidden_layer_sizes=97, score=0.6210374639769453, total= 6.8min
[CV] . hidden_layer_sizes=457, score=0.6474820143884892, total=12.7min
[CV] . hidden_layer_sizes=717, score=0.6355810616929699, total=20.1min
[CV] . hidden_layer_sizes=634, score=0.6172661870503597, total=14.4min
[CV] .

[Parallel(n_jobs=100)]: Done 500 out of 500 | elapsed: 111.1min finished


Elapsed time:
6687.72568154335


## Savel results, models and pipeline to a .pkl file 

In [14]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [15]:
experiment.save_models(random_search)
experiment.save_pipeline(data_preprocessing)